In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
import plotly.express as px

In [2]:
data = pd.read_csv("learn.csv")

In [3]:
del data["Unnamed: 0"]

In [4]:
data.loanapply_insert_time = pd.to_datetime(data.loanapply_insert_time)
data.birth_year = pd.to_datetime(data.birth_year)
data.insert_time = pd.to_datetime(data.insert_time)
data.company_enter_month = pd.to_datetime(data.company_enter_month)

In [5]:
x = data.copy()

In [6]:
c1 = pd.get_dummies(x.income_type)
c2 = pd.get_dummies(x.employment_type)
c2.rename(columns = {"기타":"기타_직업"}, inplace = True)
c3 = pd.get_dummies(x.houseown_type)
c4 = pd.get_dummies(x.purpose)
c4.rename(columns = {"기타": "기타_목적"}, inplace = True)

In [7]:
c = pd.concat([c1, c2, c3, c4], axis = 1)
x.drop(["income_type", "employment_type", "houseown_type", "purpose"], inplace = True, axis= 1)
x = pd.concat([x, c], axis = 1)

In [8]:
test_data=x[x['insert_time']>='2022-05-16']

In [9]:
train_data = x[x['insert_time']<'2022-05-16']

In [10]:
test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13271/4276231571.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  test_data = test_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [11]:
test_label = test_data["is_applied"]
del test_data["is_applied"]

In [12]:
train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13271/1691262433.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.reset_index except for the argument 'level' will be keyword-only.
  train_data = train_data.groupby('application_id').max().reset_index('inplace' == True, 'drop' == True)


In [13]:
train_label = train_data["is_applied"]
del train_data["is_applied"]

In [14]:
from catboost import CatBoostClassifier

In [15]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [16]:
model.fit(train_data,train_label)

0:	learn: 0.6630300	total: 100ms	remaining: 100ms
1:	learn: 0.6579416	total: 139ms	remaining: 0us


In [17]:
from sklearn.metrics import f1_score

In [18]:
p = model.predict(test_data)

In [19]:
f1_score(test_label, p)

0.6609089731969441

In [20]:
app_yes_index = test_data.application_id[p==1]

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
lr = LogisticRegression(class_weight = 'balanced')

In [23]:
y = data.copy()

In [24]:
test_d = y[y["insert_time"] >= '2022-05-16']

In [25]:
train_d = y[y['insert_time']<'2022-05-16']

In [26]:
test_l = test_d['is_applied']
train_l = train_d['is_applied']

In [27]:
test_d = test_d[['loan_rate', 'loan_limit']]
train_d = train_d[['loan_rate', 'loan_limit']]

In [28]:
test_d['loan_limit'] = np.log10(1+test_d['loan_limit'])
train_d['loan_limit'] = np.log10(1+train_d['loan_limit'])

In [29]:
lr.fit(train_d, train_l)

LogisticRegression(class_weight='balanced')

In [30]:
p2 = lr.predict(test_d)

In [31]:
f1_score(test_l, p2)

0.1335115694758134

In [32]:
z = data.copy()

In [33]:
z["predict"] = 0

In [43]:
z = z.loc[z.insert_time > "2022-05-16"]

In [44]:
yes = z.loc[z.application_id.isin(app_yes_index)]

In [45]:
yes["score"] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13271/3098435085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yes["score"] = 0


In [46]:
log_lr = np.log10(1+yes.loan_limit)

In [47]:
xxx = pd.concat([yes.loan_rate, log_lr], axis = 1)

In [48]:
yes["score"] = lr.predict_proba(xxx)

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_13271/729384516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yes["score"] = lr.predict_proba(xxx)


In [49]:
yesyes = yes.copy()

In [50]:
k = yesyes.sort_values(by = 'score').drop_duplicates("application_id", keep = "last").index

In [52]:
len(k)

81926

In [58]:
z.loc[k, "predict"] = 1

In [62]:
z["predict2"] = 0

In [63]:
kk = yesyes.sort_values(by = "loan_rate").drop_duplicates("application_id", keep = "last").index

In [64]:
z.loc[kk, "predict2"] = 1

In [65]:
z

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,predict,predict2
8395658,2136969,2022-05-16 00:11:51,13,123,13000000.0,17.5,0.0,355550,1994-01-01,1.0,2022-05-16 00:11:50,650.0,24000000.0,OTHERINCOME,2022-06-01,기타,기타가족소유,2000000.0,생활비,0.0,0.0,6.0,10000000.0,1,1
8395659,2136969,2022-05-16 00:11:51,35,29,5000000.0,14.5,0.0,355550,1994-01-01,1.0,2022-05-16 00:11:50,650.0,24000000.0,OTHERINCOME,2022-06-01,기타,기타가족소유,2000000.0,생활비,0.0,0.0,6.0,10000000.0,0,0
8395660,2136969,2022-05-16 00:11:52,21,196,3000000.0,12.5,1.0,355550,1994-01-01,1.0,2022-05-16 00:11:50,650.0,24000000.0,OTHERINCOME,2022-06-01,기타,기타가족소유,2000000.0,생활비,0.0,0.0,6.0,10000000.0,0,0
8395661,824356,2022-05-16 00:11:53,22,221,10000000.0,17.5,1.0,102740,1982-01-01,1.0,2022-05-16 00:11:51,630.0,35000000.0,EARNEDINCOME,2020-10-01,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,6.0,92000000.0,1,1
8395662,993637,2022-05-16 00:12:15,35,29,5000000.0,16.3,1.0,707400,1982-01-01,1.0,2022-05-16 00:12:14,700.0,2000000.0,OTHERINCOME,2022-06-01,기타,기타가족소유,1000000.0,생활비,0.0,0.0,2.0,1000000.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264381,455157,2022-05-31 23:54:28,42,258,3000000.0,18.0,0.0,717233,1973-01-01,1.0,2022-05-31 23:54:27,610.0,106000000.0,EARNEDINCOME,1997-04-01,정규직,자가,50000000.0,대환대출,0.0,0.0,12.0,293000000.0,0,0
10264382,455157,2022-05-31 23:54:28,46,227,3000000.0,17.5,0.0,717233,1973-01-01,1.0,2022-05-31 23:54:27,610.0,106000000.0,EARNEDINCOME,1997-04-01,정규직,자가,50000000.0,대환대출,0.0,0.0,12.0,293000000.0,0,0
10264383,455157,2022-05-31 23:54:29,22,221,5000000.0,18.4,0.0,717233,1973-01-01,1.0,2022-05-31 23:54:27,610.0,106000000.0,EARNEDINCOME,1997-04-01,정규직,자가,50000000.0,대환대출,0.0,0.0,12.0,293000000.0,0,0
10264384,455157,2022-05-31 23:54:37,38,16,3000000.0,14.5,0.0,717233,1973-01-01,1.0,2022-05-31 23:54:27,610.0,106000000.0,EARNEDINCOME,1997-04-01,정규직,자가,50000000.0,대환대출,0.0,0.0,12.0,293000000.0,0,0


In [61]:
f1_score(z.is_applied, z.predict)

0.1962658415631755

In [66]:
f1_score(z.is_applied, z.predict2)

0.17542279489679144